# 10. Concurrent Workflows with Agent Framework

## 개요
이 노트북에서는 Microsoft Agent Framework의 **Concurrent Workflow** 패턴을 학습합니다. 동시 실행 워크플로우를 통해 여러 에이전트가 병렬로 작업을 수행하고, 결과를 통합하는 Fan-out/Fan-in 아키텍처를 구현해봅니다.

## 학습 목표
- **Concurrent Workflow** 개념 이해
- 여러 전문 에이전트의 병렬 실행
- Fan-out/Fan-in 패턴 구현
- 워크플로우를 단일 에이전트로 래핑하는 방법
- 병렬 처리 결과의 집계 및 관리

## 주요 특징
- **병렬 실행**: 여러 에이전트가 동시에 작업 수행
- **전문성 활용**: 각 에이전트가 특정 영역에 특화
- **결과 통합**: 모든 에이전트의 출력을 하나로 집계
- **재사용성**: 전체 워크플로우를 단일 에이전트처럼 사용

## 1. Import Required Libraries

필요한 라이브러리들을 가져옵니다.

In [21]:
import asyncio
import logging
from typing import List, Dict, Any
import os

# Agent Framework 관련 imports
from agent_framework import ConcurrentBuilder
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
from dotenv import load_dotenv

from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv(override=True)

# 환경 변수 확인
print("환경 변수 확인:")
env_vars = [
    "AZURE_OPENAI_ENDPOINT",
    "AZURE_OPENAI_API_KEY",
    "AZURE_OPENAI_API_VERSION",
    "AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"
]

for var in env_vars:
    value = os.getenv(var)
    if value:
        # API 키는 부분적으로만 표시
        if "API_KEY" in var:
            print(f"✅ {var}: {value[:8]}...")
        else:
            print(f"✅ {var}: {value}")
    else:
        print(f"❌ {var}: 설정되지 않음")

print("\n💡 환경 변수가 설정되지 않은 경우, Azure CLI 인증을 사용합니다.")



환경 변수 확인:
✅ AZURE_OPENAI_ENDPOINT: https://aistudioaiservices343274518006.openai.azure.com
✅ AZURE_OPENAI_API_KEY: 6hBhIb88...
✅ AZURE_OPENAI_API_VERSION: 2024-02-01
✅ AZURE_OPENAI_CHAT_DEPLOYMENT_NAME: gpt-4.1

💡 환경 변수가 설정되지 않은 경우, Azure CLI 인증을 사용합니다.


## 2. Setup Logging Configuration

워크플로우 실행과 에이전트 상호작용을 추적하기 위한 로깅을 설정합니다.

In [22]:
# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler('concurrent_workflow.log')
    ]
)

logger = logging.getLogger(__name__)

def log_agent_activity(agent_name: str, activity: str, details: str = ""):
    """에이전트 활동을 로그로 기록"""
    logger.info(f"🤖 [{agent_name}] {activity}: {details}")

def log_workflow_event(event_type: str, details: str = ""):
    """워크플로우 이벤트를 로그로 기록"""
    logger.info(f"🔄 [WORKFLOW] {event_type}: {details}")

print("✅ 로깅 설정 완료")

✅ 로깅 설정 완료


## 3. Create Team Members (Agents)

서로 다른 역할과 전문성을 가진 여러 에이전트를 생성합니다. 각 에이전트는 특정 도메인에 특화되어 있으며, 동시에 작업을 수행할 수 있습니다.

### 팀 구성:
- **Research Agent**: 시장 조사 및 제품 분석 전문가
- **Marketing Agent**: 마케팅 전략 및 메시지 전문가  
- **Legal Agent**: 법무 및 컴플라이언스 검토 전문가

In [23]:
async def create_specialized_agents():
    """전문화된 에이전트들을 생성합니다."""
    
    # Azure OpenAI 클라이언트 생성
    log_workflow_event("INITIALIZATION", "Creating Azure OpenAI client")
    chat_client = AzureOpenAIChatClient(credential=AzureCliCredential())
    
    # 1. Research Agent - 시장 조사 및 분석 전문가
    researcher = chat_client.create_agent(
        instructions=(
            "당신은 전문 시장 조사 및 제품 분석가입니다. "
            "주어진 프롬프트에 대해 간결하고 사실적인 통찰력, 기회, 위험을 제공하세요. "
            "시장 트렌드, 경쟁 환경, 타겟 고객에 대한 데이터 기반 분석을 포함하세요."
        ),
        name="researcher",
    )
    log_agent_activity("researcher", "CREATED", "Market research and analysis specialist")
    
    # 2. Marketing Agent - 마케팅 전략 전문가
    marketer = chat_client.create_agent(
        instructions=(
            "당신은 창의적인 마케팅 전략가입니다. "
            "프롬프트에 맞는 매력적인 가치 제안과 타겟 메시징을 개발하세요. "
            "브랜딩, 포지셔닝, 고객 세그먼트에 따른 맞춤형 메시지를 제공하세요."
        ),
        name="marketer",
    )
    log_agent_activity("marketer", "CREATED", "Creative marketing strategist")
    
    # 3. Legal Agent - 법무 및 컴플라이언스 전문가
    legal = chat_client.create_agent(
        instructions=(
            "당신은 신중한 법무/컴플라이언스 검토 전문가입니다. "
            "프롬프트를 기반으로 제약사항, 면책조항, 정책 우려사항을 강조하세요. "
            "규제 준수, 안전 요구사항, 책임 문제를 중점적으로 다루세요."
        ),
        name="legal",
    )
    log_agent_activity("legal", "CREATED", "Legal and compliance reviewer")
    
    print("✅ 전문 에이전트 팀 생성 완료")
    print(f"   - 📊 Research Agent: 시장 조사 전문가")
    print(f"   - 📈 Marketing Agent: 마케팅 전략가") 
    print(f"   - ⚖️ Legal Agent: 법무 컴플라이언스")
    
    return researcher, marketer, legal

# 에이전트 생성 (비동기 실행을 위해 이후 셀에서 실행)
print("🚀 에이전트 생성 함수 정의 완료")

🚀 에이전트 생성 함수 정의 완료


## 4. Define the Concurrent Workflow

ConcurrentBuilder를 사용하여 여러 에이전트가 동시에 작업할 수 있는 워크플로우 구조를 설정합니다.

### Concurrent Workflow 특징:
- **Fan-out**: 입력을 여러 에이전트에 동시 전달
- **Parallel Processing**: 각 에이전트가 독립적으로 작업 수행
- **Fan-in**: 모든 에이전트의 결과를 하나로 집계
- **Agent Wrapper**: 전체 워크플로우를 단일 에이전트처럼 사용

In [24]:
async def create_concurrent_workflow():
    """동시 실행 워크플로우를 생성합니다."""
    
    log_workflow_event("SETUP", "Creating concurrent workflow")
    
    # 1. 전문 에이전트들 생성
    researcher, marketer, legal = await create_specialized_agents()
    
    # 2. ConcurrentBuilder로 워크플로우 구성
    log_workflow_event("BUILDING", "Configuring concurrent workflow with participants")
    
    workflow = ConcurrentBuilder().participants([
        researcher, 
        marketer, 
        legal
    ]).build()
    
    # 3. 워크플로우를 단일 에이전트로 래핑
    log_workflow_event("WRAPPING", "Converting workflow to agent interface")
    
    concurrent_agent = workflow.as_agent(name="ConcurrentWorkflowAgent")
    
    print("✅ 동시 실행 워크플로우 생성 완료")
    print("   📋 구성:")
    print("   ├── 🔄 ConcurrentBuilder: 병렬 실행 관리자")
    print("   ├── 👥 Participants: 3개 전문 에이전트")
    print("   └── 🤖 Agent Wrapper: 단일 인터페이스 제공")
    
    return concurrent_agent, workflow

# 워크플로우 생성 함수 정의 완료
print("🏗️ 워크플로우 생성 함수 정의 완료")

🏗️ 워크플로우 생성 함수 정의 완료


## 5. Execute the Workflow

동시 실행 워크플로우를 실행하여 여러 에이전트가 협력하고 병렬로 작업을 수행하는 과정을 시연합니다.

### 실행 시나리오
도시 통근자를 위한 저렴한 전기 자전거 출시에 대해 각 전문가의 관점에서 동시에 분석해봅시다.

In [25]:
async def execute_concurrent_analysis():
    """동시 실행 워크플로우를 실행합니다."""
    
    print("🚀 동시 실행 워크플로우 시작")
    print("=" * 60)
    
    # 1. 워크플로우 생성
    concurrent_agent, workflow = await create_concurrent_workflow()
    
    # 2. 분석할 비즈니스 시나리오
    prompt = """
    우리는 도시 통근자를 위한 저렴한 전기 자전거를 출시하려고 합니다. 
    
    제품 특징:
    - 합리적인 가격대의 전기 자전거
    - 도시 환경에 최적화
    - 일상 통근용도로 설계
    - 친환경적이고 효율적인 교통수단
    
    각각의 전문가 관점에서 이 제품 출시에 대해 분석해주세요.
    """
    
    # 3. 워크플로우 실행 (병렬 처리)
    log_workflow_event("EXECUTION_START", f"Running concurrent analysis")
    print(f"\n📋 분석 주제: 도시 통근용 전기 자전거 출시")
    print(f"🔄 병렬 실행 중... (Research + Marketing + Legal)")
    
    try:
        # 동시 실행 - 모든 에이전트가 병렬로 작업
        import time
        start_time = time.time()
        
        agent_response = await concurrent_agent.run(prompt)
        
        execution_time = time.time() - start_time
        log_workflow_event("EXECUTION_COMPLETE", f"Completed in {execution_time:.2f}s")
        
        print(f"\n✅ 병렬 실행 완료 (소요시간: {execution_time:.2f}초)")
        
        return agent_response
        
    except Exception as e:
        log_workflow_event("EXECUTION_ERROR", f"Error: {str(e)}")
        print(f"❌ 실행 오류: {str(e)}")
        return None

# 실행 함수 정의 완료
print("⚡ 워크플로우 실행 함수 정의 완료")

⚡ 워크플로우 실행 함수 정의 완료


## 6. Handle Workflow Results

동시 실행의 결과를 처리하고 표시합니다. 각 에이전트의 출력이 어떻게 수집되고 관리되는지 보여줍니다.

In [26]:
def display_concurrent_results(agent_response):
    """동시 실행 결과를 분석하고 표시합니다."""
    
    if not agent_response or not agent_response.messages:
        print("❌ 결과가 없습니다.")
        return
    
    print("\n" + "=" * 80)
    print("📊 동시 실행 워크플로우 결과 분석")
    print("=" * 80)
    
    # 메시지 통계
    total_messages = len(agent_response.messages)
    agents_participated = set()
    
    print(f"\n📈 실행 통계:")
    print(f"   총 메시지 수: {total_messages}")
    
    # 각 메시지 분석 및 표시
    for i, msg in enumerate(agent_response.messages, start=1):
        role = getattr(msg.role, "value", msg.role) if hasattr(msg, 'role') else "unknown"
        author = msg.author_name if hasattr(msg, 'author_name') and msg.author_name else role
        
        if author != "user":
            agents_participated.add(author)
        
        # 구분선과 헤더
        print(f"\n{'-' * 60}")
        print(f"📝 메시지 {i:02d} [{author.upper()}]")
        print(f"{'-' * 60}")
        
        # 메시지 내용 (길이에 따라 요약)
        content = msg.text if hasattr(msg, 'text') else str(msg)
        if len(content) > 500:
            print(f"{content[:500]}...")
            print(f"\n💡 [내용이 길어 축약됨 - 전체 길이: {len(content)}자]")
        else:
            print(content)
    
    # 참여 에이전트 요약
    print(f"\n\n🤖 참여한 에이전트: {len(agents_participated)}개")
    for agent in sorted(agents_participated):
        print(f"   ✓ {agent}")
    
    return {
        "total_messages": total_messages,
        "agents_participated": len(agents_participated),
        "agent_names": list(agents_participated)
    }

def analyze_workflow_performance(stats: Dict[str, Any]):
    """워크플로우 성능을 분석합니다."""
    
    print(f"\n\n🔍 워크플로우 성능 분석")
    print(f"{'=' * 40}")
    
    print(f"✅ 병렬 처리 효과:")
    print(f"   - 동시 실행된 에이전트: {stats['agents_participated']}개")
    print(f"   - 총 생성된 응답: {stats['total_messages']}개")
    
    if stats['agents_participated'] > 1:
        print(f"   - 🚀 성공적인 병렬 처리 확인")
        print(f"   - ⚡ 단일 요청으로 다중 전문가 분석 완료")
    else:
        print(f"   - ⚠️  병렬 처리 효과 제한적")

# 결과 처리 함수 정의 완료
print("📊 결과 분석 함수 정의 완료")

📊 결과 분석 함수 정의 완료


## 실제 실행 및 데모

이제 정의한 함수들을 사용하여 실제로 동시 실행 워크플로우를 실행해봅시다.

In [27]:
# 전체 데모 실행
async def run_concurrent_workflow_demo():
    """동시 실행 워크플로우 전체 데모를 실행합니다."""
    
    print("🎯 Microsoft Agent Framework - Concurrent Workflow Demo")
    print("=" * 70)
    
    try:
        # 1. 워크플로우 실행
        response = await execute_concurrent_analysis()
        
        if response:
            # 2. 결과 분석 및 표시
            stats = display_concurrent_results(response)
            
            # 3. 성능 분석
            analyze_workflow_performance(stats)
            
            print(f"\n🎉 데모 완료!")
            print(f"   동시 실행 워크플로우가 성공적으로 실행되었습니다.")
            
        else:
            print("❌ 워크플로우 실행 실패")
            
    except Exception as e:
        print(f"❌ 데모 실행 중 오류: {str(e)}")
        logger.error(f"Demo execution failed: {str(e)}")

# 실행하려면 아래 주석을 해제하세요
await run_concurrent_workflow_demo()

print("🎬 데모 실행 준비 완료")
print("💡 실행하려면: await run_concurrent_workflow_demo()")
print("⚠️  주의: Azure OpenAI 설정과 인증이 필요합니다.")

🎯 Microsoft Agent Framework - Concurrent Workflow Demo
🚀 동시 실행 워크플로우 시작
✅ 전문 에이전트 팀 생성 완료
   - 📊 Research Agent: 시장 조사 전문가
   - 📈 Marketing Agent: 마케팅 전략가
   - ⚖️ Legal Agent: 법무 컴플라이언스
✅ 동시 실행 워크플로우 생성 완료
   📋 구성:
   ├── 🔄 ConcurrentBuilder: 병렬 실행 관리자
   ├── 👥 Participants: 3개 전문 에이전트
   └── 🤖 Agent Wrapper: 단일 인터페이스 제공

📋 분석 주제: 도시 통근용 전기 자전거 출시
🔄 병렬 실행 중... (Research + Marketing + Legal)

✅ 병렬 실행 완료 (소요시간: 21.17초)

📊 동시 실행 워크플로우 결과 분석

📈 실행 통계:
   총 메시지 수: 3

------------------------------------------------------------
📝 메시지 01 [RESEARCHER]
------------------------------------------------------------
**1. 시장 트렌드 분석**

- **도시화 및 친환경 교통 수요 증대**: 도시 인구 증가와 교통 혼잡, 미세먼지 등 환경 이슈로 친환경·개인 맞춤형 교통수단에 대한 관심이 높아지고 있습니다.
- **전기 자전거 성장세**: 글로벌 e-자전거 시장은 연평균 10% 이상 성장(2023 기준 $40B 이상), 특히 합리적 가격의 도시형 모델 수요가 급증(중국, 미국, 유럽 주요 도시 중심).
- **정부 지원 확대**: 일부 국가/도시에서는 전기 자전거 구매 보조금, 인프라 확대 등 정책 지원이 활발합니다.

**2. 경쟁 환경 분석**

- **주요 경쟁사**: 샤오미, GOGOBEST(중국), Bianchi(이탈리아), 플랫폼 기반 브랜드(Yadea, Rad Power 등) 등 다양한

## 심화 학습: 커스텀 동시 실행 패턴

여러분만의 동시 실행 시나리오를 만들어보세요!

In [28]:
# 💡 연습 과제: 다른 도메인의 전문가 팀 만들기

async def create_custom_expert_team():
    """여러분만의 전문가 팀을 만들어보세요!"""
    
    # TODO: 다른 도메인의 전문가들로 팀을 구성해보세요
    # 예시 아이디어:
    # - 기술 전문가 (Technical Specialist)
    # - 재무 분석가 (Financial Analyst)  
    # - 사용자 경험 전문가 (UX Specialist)
    # - 환경 영향 평가자 (Environmental Impact Assessor)
    
    print("🎯 연습 과제: 여러분만의 전문가 팀을 만들어보세요!")
    print("\n아이디어:")
    print("1. 다양한 전문 분야의 에이전트 생성")
    print("2. 새로운 비즈니스 시나리오 설정")
    print("3. 각 전문가의 고유한 관점 정의")
    print("4. 동시 실행으로 다각도 분석 수행")
    
    # 여기에 여러분의 코드를 작성하세요!

# 실험해보세요!
print("🔬 실험용 함수 준비 완료")
print("📝 위의 TODO 부분을 수정하여 나만의 전문가 팀을 만들어보세요!")

🔬 실험용 함수 준비 완료
📝 위의 TODO 부분을 수정하여 나만의 전문가 팀을 만들어보세요!


## 📚 학습 정리 및 핵심 개념

### 🔑 핵심 개념 요약

**1. Concurrent Workflow란?**
- 여러 에이전트가 **동시에** 작업을 수행하는 워크플로우 패턴
- **Fan-out/Fan-in** 아키텍처로 입력 분산 및 결과 집계
- 각 에이전트의 **전문성을 병렬로 활용**

**2. 주요 구성 요소**
- `ConcurrentBuilder`: 병렬 실행 워크플로우 생성기
- `participants()`: 동시 실행할 에이전트들 등록
- `workflow.as_agent()`: 워크플로우를 단일 에이전트로 래핑

**3. 장점과 활용 사례**
- ✅ **성능 향상**: 순차 실행 대비 시간 단축
- ✅ **전문성 활용**: 각 도메인별 특화된 분석
- ✅ **확장성**: 새로운 전문가 추가 용이
- ✅ **재사용성**: 워크플로우를 하나의 에이전트처럼 사용

### 🎯 다음 단계
- 다양한 도메인의 전문가 조합 실험
- 워크플로우 성능 최적화 방법 탐구  
- 오류 처리 및 장애 복구 메커니즘 학습
- 실시간 모니터링 및 로깅 시스템 구축

### 📖 추가 학습 자료
- [Microsoft Agent Framework Documentation](https://github.com/microsoft/agent-framework)
- [Concurrent Programming Patterns](https://docs.python.org/3/library/asyncio.html)
- [Azure OpenAI Integration Guide](https://docs.microsoft.com/azure/cognitive-services/openai/)